Rename a bunch of files using Git and renamer?



In [ ]:
var execSync = require('child_process').execSync;
try {
    require.resolve('renamer');
} catch (e) {
    execSync('npm install renamer');
}
var renamer = require('renamer');
var renameUsingGit = (root, match, find, replace) => {
    var files = renamer.expand(path.join(root, match));
    var results = renamer.replace({
        files: files.filesAndDirs, 
        find: find, 
        replace: replace});
    return renamer.dryRun(results).list.map(r => {
        // rename with git instead
        var output = execSync('git ' 
                 + '--work-tree=' + JSON.stringify(root)
                 + ' --git-dir=' + JSON.stringify(path.join(root, '.git'))
                 + ' mv ' + JSON.stringify(r.before) 
                 + ' ' + JSON.stringify(r.after));
        return output.toString() || r.after;
    });
};
renameUsingGit;


In [ ]:
// test it
// rename all scss files to less
//renameUsingGit(
//    '/Users/briancullinan/Documents/portal',
//    'src/**/*.scss',
//    '.scss',
//    '.less');

How to use memory-fs and rewire to audit cli events?



In [ ]:
var path = require('path');
var execSync = require('child_process').execSync;
try {
    require.resolve('mock-require');
    require.resolve('unionfs');
    require.resolve('typescript');
    require.resolve('memfs');
} catch (e) {
    execSync('npm install mock-require memfs typescript unionfs');
}
// TODO: use https://github.com/jhnns/rewire for replacing variables, something like https://github.com/mariocasciaro/object-path?

var mockTypescriptFs = (root, data) => {
    var mock = require('mock-require');
    var memfs = require('memfs');
    var mem = new memfs.Volume;
    var unionfs = require('unionfs');
    var fs = require('fs');
    
    mem.mountSync(root, data);

    // Create a union of two file systems:
    unionfs
        .use(fs)
        .use(mem)
        .replace(fs);
    
    var ts = require('typescript');
    var tsMock = Object.assign({}, ts);
    var toMock = {
        readFile: 'readFileSync',
        realpath: 'realpathSync',
        writeFile: 'writeFileSync',
        fileExists: 'existsSync',
        directoryExists: 'existsSync',
        createDirectory: 'mkdirSync',
        getDirectories: 'readdirSync'
    };
    Object.keys(toMock)
        .forEach(k => {
            if(typeof mem[toMock[k]] === 'function') {
                tsMock.sys[k] = function () {
                    // typescript.sys functions return defined on error
                    try {
                        return mem[toMock[k]].apply(mem, arguments);
                    } catch (e) {
                        return;
                    }
                };
            }
        });

    mock('typescript', tsMock);
    return mock;
};
mockTypescriptFs;

// TODO: search github for projects like graceful-fs and then search google for projects that use graceful-fs and see if we can record some filesystem activity



In [ ]:

//mockTypescriptFs();


Parse typescript includes for an entire project?



In [ ]:
var importer = require('../Core');
var path = require('path');
var execSync = require('child_process').execSync;
try {
    require.resolve('gulp');
    require.resolve('gulp-tap');
} catch (e) {
    execSync('npm install "github:gulpjs/gulp.git#4.0" gulp-tap');
}
var gulp = require('gulp');
var tap = require('gulp-tap');

var projectWordCloud = (project) => {
    var words = [];
    gulp.task('get all imports', function() {
        return gulp.src(['src/**/*.ts'], {cwd: project})
            .pipe(tap(function (file) {
                var content = file.contents.toString();
                var re = /from\s+.*?["'](.*?)['"]/ig;
                var m;
                while ((m = re.exec(content))) {
                    words[words.length] = m[1];
                }
            }));
    });

    var wordCount = function (words) {
        var wordCount = {};
        words.forEach(w => {
            var word = w.split('/').pop();
            if(typeof wordCount[word] == 'undefined') {
                wordCount[word] = 15;
            } else {
                wordCount[word]++;
            }
        });
        return Object.keys(wordCount).map(function(d) {
             return {text: d, size: wordCount[d]};
        });
    };
    
    return importer.interpretAll(['gulp task to promise'])
        .then(r => {
            var tasksToPromise = eval(r[0].code);
            return tasksToPromise(['get all imports'], []);
        })
        .then(() => importer.importNotebook('../Frameworks/d3.ipynb'))
        .then(r => {
            var d3CloudToSVG = r.d3CloudToSVG;
            return d3CloudToSVG(wordCount(words));
        });
};
projectWordCloud;



In [ ]:
// TODO: create a gulp stream angular component out of it
// TODO: connect with gulp-git https://www.npmjs.com/package/gulp-git
// install https://github.com/Automattic/node-canvas/wiki/installation---osx
// install using brew install pkg-config fontconfig cairo pixman giflib libjpeg libpng
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/portal';

$$.async()
projectWordCloud(project)
    .then(svg => $$.svg(svg))
    .catch(e => $$.sendError(e));



In [1]:
var importer = require('../Core');
var execSync = require('child_process').execSync;
try {
    require.resolve('glob');
    require.resolve('gulp');
    require.resolve('gulp-replace');
    require.resolve('gulp-tap');
} catch (e) {
    execSync('npm install glob "github:gulpjs/gulp.git#4.0" gulp-replace gulp-tap');
}
var fs = require('fs');
var path = require('path');
var glob = require('glob');
var gulp = require('gulp');
var tap = require('gulp-tap');
var replace = require('gulp-replace');

// list all broken references in modules?
var projectRelatives = (project) => {
    var relatives = [];
    gulp.task('get relative links', function() {
        return gulp.src(['src/**/*.ts'], {cwd: project})
            .pipe(tap(function (file) {
                var content = file.contents.toString();
                var re = /from\s+.*?["'](.*?)['"]/ig;
                var m;
                while ((m = re.exec(content))) {
                    if(m[1].match(/^\./)) {
                        relatives[relatives.length] = file.path + '/' + m[1];
                    }
                }
            }));
    });
    
    var findSimilarFile = (m) => {
        var file = m.split('.ts/')[0];
        var fromPath = m.split('.ts/')[1];
        return new Promise((resolve, reject) => {
            var searchStrings = '**/' + path.basename(path.dirname(m)) + '/' + path.basename(m) + '.ts';
            console.log('Search for ' + searchStrings);
            glob(searchStrings, {exclude: ['**/node_modules/**'], cwd: project}, function (err, matches) {
                if(err) {
                    return reject(err);
                } else if (matches.length == 0) {
                    return resolve();
                }
                
                // TODO: try to build and make sure it works before patching
                var newRelative = path.relative(path.dirname(m), path.join(project, matches[0])).replace('.ts', '');
                
                gulp.task('fix include', function() {
                    var fixFile = path.relative(project, file);
                    return gulp.src(['**/' + fixFile + '.ts'], {cwd: project})
                        .pipe(replace(fromPath, newRelative))
                        .pipe(gulp.dest(project));
                });
        
                resolve(newRelative);
            });
        })
        .then(r => m + '->' + r);
    };
    
    return importer.interpretAll(['gulp task to promise'])
        .then(r => {
            var tasksToPromise = eval(r[0].code);
            return tasksToPromise(['get relative links'], []);
        })
        .then(() => relatives.filter(r => {
            var file = r.split('.ts/')[0];
            var fromPath = r.split('.ts/')[1];
            var res = path.resolve(path.join(path.dirname(file), fromPath));
            return !fs.existsSync(res + '.ts');
        }))
        .then(missing => Promise.all(missing.map(m => findSimilarFile(m)).filter(e => typeof e != 'undefined')))
        .then(() => importer.interpretAll(['gulp task to promise']))
        .then(r => {
            var tasksToPromise = eval(r[0].code);
            return tasksToPromise(['fix include'], []);
        });
    // TODO: try building with all results resolved
    // TODO: create path and apply
};
projectRelatives;


[Function: projectRelatives]

In [ ]:

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/portal';

$$.async()
projectRelatives(project)
    .then(svg => $$.sendResult(svg))
    .catch(e => $$.sendError(e));
   


In [ ]:
var execSync = require('child_process').execSync;
try {
    require.resolve('gulp');
    require.resolve('gulp-watch');
} catch (e) {
    execSync('npm install "github:gulpjs/gulp.git#4.0" gulp-watch');
}
var gulp = require('gulp'),
    watch = require('gulp-watch');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/portal';

$$.async();
var searching = false;;
gulp.task('callback', function () {
    // Callback mode, useful if any plugin in the pipeline depends on the `end`/`flush` event 
    return watch('src/**/*.ts', { cwd: project, ignoreInitial: true, read: false, readDelay: 100 }, function () {
        if(!searching) {
            searching = true;
            projectRelatives(project)
                .then(() => { searching = false });
        }
    });
});

gulp.task(['callback'])();



Search for **/config/environment.ts
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Search for **/config/environment.ts
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Search for **/config/environment.ts
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
